In [2]:
'''
tracking the wrist and have them in a frame
get to see the difference image formed within these windows

'''

path = r"C:\Users\ayush\Desktop\Jump Rope Code\Media\JumpRopeChairAngle_Trim.mp4"
path1 = r"C:\Users\ayush\Desktop\Jump Rope Code\Media\housejumpcut.mp4"
path2 = r"C:\Users\ayush\Desktop\Jump Rope Code\JumpRope\v_JumpRope_g05_c03.avi"
path3 = r"C:\Users\ayush\Desktop\Jump Rope Code\JumpRope\v_JumpRope_g24_c01.avi"
path4 = r"C:\Users\ayush\Desktop\Jump Rope Code\JumpRope\v_JumpRope_g17_c05.avi"
path5 = r"C:\Users\ayush\Desktop\Jump Rope Code\JumpRope\v_JumpRope_g08_c01.avi"
path6 = r"C:\Users\ayush\Desktop\Jump Rope Code\JumpRope\v_JumpRope_g02_c01.avi"
pathknee = r"C:\Users\ayush\Desktop\Jump Rope Code\Media\KneeJumps.mp4"
PathDoubleUnder = r"C:\Users\ayush\Desktop\Fyp Videos\Videos wb\Double Unders.mp4"
PathJumpingJacks = r"C:\Users\ayush\Desktop\Fyp Videos\Videos wb\ActualJumpingJacks.mp4"
PathNoRope = r"C:\Users\ayush\Desktop\Fyp Videos\Videos wb\NoRope.mp4"
PathJumpingJacksMain = r"C:\Users\ayush\Desktop\Fyp Videos\Videos wb\JumpingJacksMain.mp4"
PathCrissCross = r"C:\Users\ayush\Desktop\Fyp Videos\Videos wb\crisscross480.mp4"
PathTestJJ1 = r'C:\Users\ayush\Desktop\dissertation\Testing Videos\Jumping Jack 1.mp4'
PathTestJJ2 = r'C:\Users\ayush\Desktop\dissertation\Testing Videos\Jumping Jack 2.mp4'
PathTestJJ3 = r'C:\Users\ayush\Desktop\dissertation\Testing Videos\Jumping Jack 3.mp4'
PathTestHK1 = r'C:\Users\ayush\Desktop\dissertation\Testing Videos\High Knee 1.mp4'
PathTestHK2 = r'C:\Users\ayush\Desktop\dissertation\Testing Videos\High Knee 2.mp4'
PathTestHK3 = r'C:\Users\ayush\Desktop\dissertation\Testing Videos\High Knee 3.mp4'
PathTestHK4 = r'C:\Users\ayush\Desktop\dissertation\Testing Videos\High Knee 4.mp4'
PathTestNJ1 = r'C:\Users\ayush\Desktop\dissertation\Testing Videos\Normal Jump 1.mp4'
PathTestNJ2 = r'C:\Users\ayush\Desktop\dissertation\Testing Videos\Normal Jump 2.mp4'
PathTestMixed1 = r'C:\Users\ayush\Desktop\dissertation\Testing Videos\Mixed1.mp4'
PathTestMixed2 = r'C:\Users\ayush\Desktop\dissertation\Testing Videos\Mixed2.mp4'


## Imports
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tkinter as tk
from tkinter import filedialog
import time
import random
import mediapipe as mp
import numpy as np
import warnings

root = tk.Tk()
root.withdraw()

warnings.filterwarnings("ignore")

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

left_hip_x = []
right_hip_x = []
right_knee_x = []
left_knee_x = []
right_knee_y = []
left_knee_y = []
right_ankle_x = []
left_ankle_x = []
right_ankle_y = []
left_ankle_y = []
right_hip_y = []
left_hip_y = []
left_wrist_y = []
left_wrist_x = []
right_wrist_x = []
right_wrist_y = []
differenc_wrist_r = []
difference_ankle_r = []
left_foot_y = []
right_foot_y = []
left_heel_y = []
right_heel_y = []

cxlist_roi1 = []
cylist_roi1 = []
cxlist_roi2 = []
cylist_roi2 = []

CrissCrossCounter = 0
CrissCrossPosition = None

JumpingJackCounter = 0
PelvicPosition = None

Knee_counter = 0
KneePosition = None

nj_frame_list = [] 

def count_major_local_minimas(data):
    # Convert data to a numpy array
    data = np.array(data)

    # Initialize the count of major local minima to zero
    major_minima_count = 0

    # Loop through each data point and check if it's a local minimum
    for i in range(1, len(data) - 1):
        if data[i] < data[i - 1] and data[i] < data[i + 1]:
            # Calculate the difference between the current data point and its neighboring points
            diff_left = abs(data[i] - data[i - 1])
            diff_right = abs(data[i] - data[i + 1])
            diff = max(diff_left, diff_right)

            # If the difference is large enough, count the local minimum as a major one
            if diff >= 0.003:  # adjust this threshold value as per your requirement
                major_minima_count += 1

    return major_minima_count


def differencefunc(a):
    diff = []
    for i in range(len(a)):
        diff.append(a[i] - a[i - 1])
    return diff

def calculate_angle(a, b, c):
    # Convert input points to numpy arrays
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point
    c = np.array(c)  # Last point

    # Calculate the angle in radians between the lines connecting b to a and b to c
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    
    # Convert angle from radians to degrees
    angle = np.abs(radians * 180.0 / np.pi)

    # Make sure angle is not greater than 180 degrees
    if angle > 180.0:
        angle = 360 - angle

    return angle

file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi")])


knee_angle_list = []
cap = cv2.VideoCapture(file_path)

if (cap.isOpened == False):
    print("error opening video stream")

Pelvic_angle_list = []
e1_list = []
f1_list = []

m1_list = []
n1_list = []

knnfgbg = cv2.createBackgroundSubtractorKNN(history=2, dist2Threshold=700, detectShadows=False)

frameMultiple = 0
frame_count = 0
check_interval = 50

flag = False

num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

DefaultFrameNJ = [False]*num_frames
DefaultFrameJJ = [False]*num_frames
DefaultFrameCR = [False]*num_frames
DefaultFrameHK = [False]*num_frames


fps = cap.get(cv2.CAP_PROP_FPS)



## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():

        ret, frame = cap.read()

        # Recolor image to RGB

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        frame_number = cap.get(cv2.CAP_PROP_POS_FRAMES)
        
        DefaultFrameNJ.append(None)
        DefaultFrameJJ.append(None)
        DefaultFrameCR.append(None)
        DefaultFrameHK.append(None)
        frameMultiple+=1
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            left_ankle_x.append(landmarks[27].x)
            right_ankle_x.append(landmarks[28].x)
            left_ankle_y.append(landmarks[27].y)
            right_ankle_y.append(landmarks[28].y)
            right_hip_y.append(landmarks[23].y)
            right_hip_x.append(landmarks[23].x)
            left_hip_y.append(landmarks[24].y)
            left_hip_x.append(landmarks[24].x)
            left_wrist_y.append(landmarks[15].y)
            left_wrist_x.append(landmarks[15].x)
            right_wrist_y.append(landmarks[16].y)
            right_wrist_x.append(landmarks[16].x)
            left_heel_y.append(landmarks[29].y)
            right_heel_y.append(landmarks[30].y)
            left_foot_y.append(landmarks[31].y)
            right_foot_y.append(landmarks[32].y)
            left_knee_y.append(landmarks[25].y)
            left_knee_x.append(landmarks[25].x)
            right_knee_y.append(landmarks[26].y)
            right_knee_x.append(landmarks[26].x)

            right_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]
            left_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]

            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                    landmarks[mp_pose.PoseLandmark.NOSE.value].y]

            Pelvic = [(left_hip[0] + right_hip[0]) / 2, (left_hip[1] + right_hip[1]) / 2]

            # caluclating the angle at the knee
            Knee_angle = calculate_angle(left_hip, left_knee, ankle)
            Knee_angle = round(Knee_angle, 2)
            knee_angle_list.append(Knee_angle)
            # Visualizing the Angle of the knee

            #             cv2.putText(image, str(Knee_angle), tuple(np.multiply(left_knee, image.shape[::-1][1:]).astype(int))
            #                         ,cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            #       ###      counting the number of jumps made when using the high knee jump techniques
            if Knee_angle < 160:
                KneePosition = "down"
            if Knee_angle > 160 and KneePosition == 'down':
                KneePosition = "up"
                Knee_counter += 1
                DefaultFrameHK[frameMultiple]= True
                

            #             calulating the angle between knees and Pelvic to find jumping jack kind of jump
            Pelvic_angle = calculate_angle(left_knee, Pelvic, right_knee)

            Pelvic_angle_list.append(Pelvic_angle)

            # visualizing the angle
            #             cv2.putText(image, str(Pelvic_angle), tuple(np.multiply(elbow, [640, 480]).astype(int)),
            #                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            #           ##  counting the number of jumps made when using the jumping jack technique
            if Pelvic_angle < 27:
                PelvicPosition = "close"
            if Pelvic_angle > 27 and PelvicPosition == "close":
                PelvicPosition = "open"
                JumpingJackCounter += 1
                DefaultFrameJJ[frameMultiple]=True

            ## counting the number of jumps made when using criss cross:
            if left_wrist[0] < right_wrist[0]:
                CrissCrossPosition = "Crissed"
            if left_wrist[0] > right_wrist[0] and CrissCrossPosition == "Crissed":
                CrissCrossCounter += 1
                CrissCrossPosition = "Crossed"
                DefaultFrameCR[frameMultiple] = True

            ### counting the number of jumps made using double under

        except:
            pass

        ### counter logic for finding the minimas at the ankles

        ankle_counter = 0

        ankle_minima_indice = []
        # Initialize a variable to keep track of the minimum value seen so far
        min_value = None

        # Get the first reading from the sensor
        readings = left_hip_y

        #         for i in range(1, len(readings)):
        #             # Get the current and previous readings
        #             current_reading = readings[i]
        #             previous_reading = readings[i-1]

        #             # If the current reading is less than the previous reading and less than the minimum value seen so far, update the minimum value
        #             if current_reading < previous_reading and (min_value is None or current_reading < min_value):
        #                 min_value = current_reading

        #             # If the current reading is greater than or equal to the previous reading and the minimum value has been updated, increment the counter and reset the minimum value
        #             elif min_value is not None:
        #                 ankle_counter += 1
        #                 min_value = None
        #                 ankle_minima_indice.append(i-1)

        ### improved logic for counting the number of jumps in which a threshold is also set in place relative to the video in check
        for i in range(1, len(readings) - 1):
            ## comparing previous values

            if readings[i] < readings[i - 1] and readings[i] < readings[i + 1]:
                ## calculating difference between current point and its neaghbouring points
                diff_left = abs(readings[i] - readings[i - 1])
                diff_right = abs(readings[i] - readings[i + 1])
                diff = max(diff_left, diff_right)

                ### setting the threshold
                if diff >= 0.003:
                    ankle_counter += 1
        

                    

# '''def count_major_local_minimas(data):c
#     # Convert data to a numpy array
#     data = np.array(data)

#     # Initialize the count of major local minima to zero
#     major_minima_count = 0

#     # Loop through each data point and check if it's a local minimum
#     for i in range(1, len(data) - 1):
#         if data[i] < data[i-1] and data[i] < data[i+1]:
#             # Calculate the difference between the current data point and its neighboring points
#             diff_left = abs(data[i] - data[i-1])
#             diff_right = abs(data[i] - data[i+1])
#             diff = max(diff_left, diff_right)

#             # If the difference is large enough, count the local minimum as a major one
#             if diff >= 0.003: # adjust this threshold value as per your requirement
#                 major_minima_count += 1

#     return major_minima_count'''

        # rectangle 1 Top left
        #         cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)

        # rectangle 2 Top Right
        #         cv2.rectangle(image,(250,0),(1380,75),(245,117,16),-1)

        # rep data (Normal)
        cv2.putText(image, "Jumps", (image.shape[1] - 130, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2,
                    cv2.LINE_AA)
        cv2.putText(image, str(ankle_counter),
                    (image.shape[1] - 40, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)

        # Rep data (knee Highs)
        cv2.putText(image, 'Knee Highs', (image.shape[1] - 195, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(image, str(Knee_counter),
                    (image.shape[1] - 40, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)

        # Rep data (Jumping Jacks)
        cv2.putText(image, 'Jumping Jacks', (image.shape[1] - 235, 92),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(image, str(JumpingJackCounter),
                    (image.shape[1] - 40, 92),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)

        # rep data (CrissCross)
        cv2.putText(image, "Criss Cross", (image.shape[1] - 195, 132), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2,
                    cv2.LINE_AA)
        cv2.putText(image, str(CrissCrossCounter * 2),
                    (image.shape[1] - 40, 132),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)

        #         #frame data
        cv2.putText(image, 'Frame', (0, 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(image, str(int(frame_number)),
                    (10, 52),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)

        # #         frame data
        #         cv2.putText(image, 'Frame', (105,12),
        #                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
        #         cv2.putText(image, str(int(frame_number)),
        #                     (90,60),
        #                     cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        ##        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        ### left hand coordinates for region of interets around wrist
        e, f = left_wrist
        efarray = np.multiply((e, f), frame.shape[::-1][1:])
        e1, f1 = int(efarray[0]), int(efarray[1])
        f1 += 20
        e1_list.append(e1)
        f1_list.append(f1)

        e2, f2 = e1 + 70, f1 + 70

        ### right hand coordinates for region of interetst around wrist
        m, n = right_wrist
        mnarray = np.multiply((m, n), frame.shape[::-1][1:])
        m1, n1 = int(mnarray[0]), int(mnarray[1])
        n1 += 20
        m1_list.append(m1)
        n1_list.append(n1)

        m2, n2 = m1 + 70, n1 + 70

        ##applying logic for finding Jumping Jacks

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

        maskKNN = knnfgbg.apply(frame)
        # Find contours
        contours, _ = cv2.findContours(maskKNN, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            # chekc if any point lies within the contours of the provided left hand:
            for point in contour:
                i, j = point[0]
                if e1 <= i <= e2 and f1 <= j <= f2:
                    flag = True
                    break
            if flag:
                break

        ### setting regions of interest below both the wrists to detect the rope

        #     #region below left hand
        #         try:

        #             cv2.rectangle(image, (e1_list[10]-20, f1_list[10]+10), (e1_list[10]+60, f1_list[10]+70), (0, 255, 0), 2)
        #         except :
        #             pass

        #     ### region below the right hand
        #         try:

        #             cv2.rectangle(image, (m1_list[10]-70, n1_list[10]+10), (m1_list[10]+10, n1_list[10]+70), (0, 255, 0), 2)
        #         except :
        #             pass

        ### applying logic for changing of the location of the stationary hands every 50 frames.
        
        frame_count += 1
        
        if frame_count == 50:
            # left hand updated position
            cv2.rectangle(image, (e1_list[frame_count - 1] - 20, f1_list[frame_count - 1] + 10),
                          (e1_list[frame_count - 1] + 60, f1_list[frame_count - 1] + 70), (0, 255, 0), 2)
            ##right hand updated position
            cv2.rectangle(image, (m1_list[frame_count - 1] - 70, n1_list[frame_count - 1] + 10),
                          (m1_list[frame_count - 1] + 10, n1_list[frame_count - 1] + 70), (0, 255, 0), 2)
            frame_count = 0  # reset8ing the value of the counter
        else:
            cv2.rectangle(image, (e1_list[frame_count - 1] - 20, f1_list[frame_count - 1] + 10),
                          (e1_list[frame_count - 1] + 60, f1_list[frame_count - 1] + 70), (0, 255, 0), 2)
            
            cv2.rectangle(image, (m1_list[frame_count - 1] - 70, n1_list[frame_count - 1] + 10),
                          (m1_list[frame_count - 1] + 10, n1_list[frame_count - 1] + 70), (0, 255, 0), 2)
        

        try:
            if flag:
                cv2.putText(image, "Flag: Rope", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                flaglist.append(flag)
            else:
                cv2.putText(image, "Flag: No Rope", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                flaglist.append(flag)
        except:
            pass

        #         try:
        #             if CrissCrossPosition == "Crossed":
        #                 cv2.putText(image, "Crossed", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        #                 flaglist.append(flag)
        #             else:
        #                 cv2.putText(image, "Crissed", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        #                 flaglist.append(flag)

        #         except:
        #             pass

        # loading new frames after the previous ones so that the flags could be reset
        _, frame = cap.read()

        # loading the next frameq
        if frame is None:
            break
        # reset the flags
        flag = False

#         cv2.imshow("knn file",maskKNN)

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

## working with y location(making these changes within the script to make EDA easy)

fix_left_ankle_y = np.multiply(left_ankle_y, frame.shape[1]).astype(int)
fix_left_wrist_y = np.multiply(left_wrist_y, frame.shape[1]).astype(int)
fix_left_knee_y = np.multiply(left_knee_y, frame.shape[1]).astype(int)
fix_left_hip_y = np.multiply(left_hip_y, frame.shape[1]).astype(int)
fix_right_ankle_y = np.multiply(right_ankle_y, frame.shape[1]).astype(int)
fix_right_wrist_y = np.multiply(right_wrist_y, frame.shape[1]).astype(int)
fix_right_knee_y = np.multiply(right_knee_y, frame.shape[1]).astype(int)
fix_right_hip_y = np.multiply(right_hip_y, frame.shape[1]).astype(int)

## getting difference of keypoint locations y

difference_left_ankle = []
for i in range(len(fix_left_ankle_y)):
    difference_left_ankle.append(fix_left_ankle_y[i] - fix_left_ankle_y[i - 1])

difference_left_wrist = []
for i in range(len(fix_left_wrist_y)):
    difference_left_wrist.append(fix_left_wrist_y[i] - fix_left_wrist_y[i - 1])

difference_right_wrist = []
for i in range(len(fix_right_wrist_y)):
    difference_right_wrist.append(fix_right_wrist_y[i] - fix_right_wrist_y[i - 1])

difference_right_ankle = []
for i in range(len(fix_right_ankle_y)):
    difference_right_ankle.append(fix_right_ankle_y[i] - fix_right_ankle_y[i - 1])

difference_right_hip = []
for i in range(len(fix_right_hip_y)):
    difference_right_hip.append(fix_right_hip_y[i] - fix_right_hip_y[i - 1])

difference_left_hip = []
for i in range(len(fix_left_hip_y)):
    difference_left_hip.append(fix_left_hip_y[i] - fix_left_hip_y[i - 1])

difference_right_knee = []
for i in range(len(fix_right_knee_y)):
    difference_right_knee.append(fix_right_knee_y[i] - fix_right_knee_y[i - 1])

difference_left_knee = []
for i in range(len(fix_left_knee_y)):
    difference_left_knee.append(fix_left_knee_y[i] - fix_left_knee_y[i - 1])

### fixing x location

fix_left_ankle_x = np.multiply(left_ankle_x, frame.shape[0]).astype(int)
fix_left_wrist_x = np.multiply(left_wrist_x, frame.shape[0]).astype(int)
fix_left_knee_x = np.multiply(left_knee_x, frame.shape[0]).astype(int)
fix_left_hip_x = np.multiply(left_hip_x, frame.shape[0]).astype(int)
fix_right_ankle_x = np.multiply(right_ankle_x, frame.shape[0]).astype(int)
fix_right_wrist_x = np.multiply(right_wrist_x, frame.shape[0]).astype(int)
fix_right_knee_x = np.multiply(right_knee_x, frame.shape[0]).astype(int)
fix_right_hip_x = np.multiply(right_hip_x, frame.shape[0]).astype(int)

## getting difference of keypoint locations x

difference_left_ankle_x = []
for i in range(len(fix_left_ankle_x)):
    difference_left_ankle_x.append(fix_left_ankle_x[i] - fix_left_ankle_x[i - 0])

difference_left_wrist_x = []
for i in range(len(fix_left_wrist_x)):
    difference_left_wrist_x.append(fix_left_wrist_x[i] - fix_left_wrist_x[i - 0])

difference_right_wrist_x = []
for i in range(len(fix_right_wrist_x)):
    difference_right_wrist_x.append(fix_right_wrist_x[i] - fix_right_wrist_x[i - 0])

difference_right_ankle_x = []
for i in range(len(fix_right_ankle_x)):
    difference_right_ankle_x.append(fix_right_ankle_x[i] - fix_right_ankle_x[i - 0])

difference_right_hip_x = []
for i in range(len(fix_right_hip_x)):
    difference_right_hip_x.append(fix_right_hip_x[i] - fix_right_hip_x[i - 1])

difference_left_hip_x = []
for i in range(len(fix_left_hip_x)):
    difference_left_hip_x.append(fix_left_hip_x[i] - fix_left_hip_x[i - 1])

difference_right_knee_x = []
for i in range(len(fix_right_knee_x)):
    difference_right_knee_x.append(fix_right_knee_x[i] - fix_right_knee_x[i - 1])

difference_left_knee_x = []
for i in range(len(fix_left_knee_x)):
    difference_left_knee_x.append(fix_left_knee_x[i] - fix_left_knee_x[i - 1])

    